# 🚀 GRPO v8: Unsloth + vLLM Optimized (LFM2-KoEn)

**Korean-English Bidirectional Translation Tuning with GRPO**

- **Base Model**: `gyung/lfm2-1.2b-koen-mt-v6.4-merged`
- **Optimization**: Unsloth + vLLM (for 2x-5x faster generation)
- **Direction**: Bidirectional (En<->Ko)

### ⚠️ Critical Notes
1. T4 GPU (16GB VRAM)에서는 메모리 관리가 중요합니다.
2. `vllm_gpu_memory_utilization = 0.5` 권장
3. 런타임 재시작이 필수입니다 (설치 후)

In [ ]:
# 1. 패키지 설치 (런타임 재시작 필요)
!pip uninstall -y numpy
!pip install unsloth vllm
!pip install -U git+https://github.com/huggingface/trl.git
!pip install peft accelerate bitsandbytes unbabel-comet
!pip install "numpy<2.0"

print("✅ 설치 완료. [런타임] -> [세션 다시 시작] 클릭")

In [ ]:
# 2. HuggingFace 로그인
from huggingface_hub import login
login()  # Colab Secrets 사용 또는 수동 입력

In [ ]:
# 3. GPU 감지 및 설정
import torch

try:
    gpu_name = torch.cuda.get_device_name(0)
except Exception:
    gpu_name = "CPU"
print(f"Detected GPU: {gpu_name}")

if "A100" in gpu_name:
    BATCH_SIZE = 16
    GRAD_ACCUM = 2
    VLLM_MEM = 0.7
elif "L4" in gpu_name:
    BATCH_SIZE = 8
    GRAD_ACCUM = 4
    VLLM_MEM = 0.6
else:  # T4 (Free Colab) or others
    BATCH_SIZE = 4
    GRAD_ACCUM = 8
    VLLM_MEM = 0.5

print(f"Configured: Batch={BATCH_SIZE}, Accum={GRAD_ACCUM}, vLLM Mem={VLLM_MEM}")

In [ ]:
# 4. Unsloth 모델 로드
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported

MODEL_ID = "gyung/lfm2-1.2b-koen-mt-v6.4-merged"  # Base 모델
OUTPUT_MODEL_ID = "YOUR_ID/your-output-model"     # ⚠️ 변경 필요!
print(f"📌 Base Model: {MODEL_ID}")

max_seq_length = 512
lora_rank = 32

# T4는 FP16 사용
gpu_name_short = torch.cuda.get_device_name(0)
if "T4" in gpu_name_short:
    dtype = torch.float16
    print("⚠️ T4 Detected: Forcing dtype=float16")
else:
    dtype = torch.float16 if not is_bfloat16_supported() else None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_ID,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

# LoRA 설정
model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,
    target_modules="all-linear",
    lora_alpha=64,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)
print("✅ Unsloth 모델 로드 완료")

In [ ]:
# 5. COMET 보상 모델 로드
try:
    from comet import download_model, load_from_checkpoint
except ImportError:
    print("⚠️ COMET 설치 중...")
    !pip install -q unbabel-comet
    from comet import download_model, load_from_checkpoint

print("⚖️ COMET 로딩...")
comet_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(comet_path)
comet_model.eval()

COMET_ON_GPU = True
if torch.cuda.is_available() and COMET_ON_GPU:
    try:
        comet_model.to("cuda")
        print("✅ COMET on GPU (Enabled)")
    except RuntimeError:
        print("⚠️ GPU 메모리 부족, COMET CPU 사용")
        comet_model.to("cpu")
else:
    comet_model.to("cpu")

def reward_function(completions, src, ref, **kwargs):
    inputs = []
    for s, mt, r in zip(src, completions, ref):
        if isinstance(mt, dict):
            mt_text = mt.get('content', str(mt))
        elif isinstance(mt, list) and len(mt) > 0:
            mt_text = mt[-1].get('content', str(mt[-1])) if isinstance(mt[-1], dict) else str(mt[-1])
        else:
            mt_text = str(mt)
        inputs.append({"src": s, "mt": mt_text, "ref": r})

    with torch.no_grad():
        gpus = 1 if torch.cuda.is_available() and COMET_ON_GPU else 0
        model_output = comet_model.predict(inputs, batch_size=len(inputs), gpus=gpus)

    return model_output.scores

In [ ]:
# 6. 데이터셋 준비
from datasets import load_dataset

DATASET_REPO = "gyung/lfm2-koen-samples"
DATASET_CONFIG = "manual_1000_grpo"  # HuggingFace 데이터셋
# 로컬 파일 사용 시: load_dataset("json", data_files="your_file.jsonl")

print(f"📚 Loading: {DATASET_REPO} ({DATASET_CONFIG})")
dataset = load_dataset(DATASET_REPO, DATASET_CONFIG, split="train")
print(f"   Sample Count: {len(dataset)}")

def Format_Bidirectional_Prompt(example):
    direction = example.get('direction', 'en2ko')

    if direction == 'ko2en':
        sys_msg = "Translate the following text to English."
    else:
        sys_msg = "Translate the following text to Korean."

    messages = [
        {"role": "system", "content": sys_msg},
        {"role": "user", "content": example['input']}
    ]

    prompt_text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    return {
        "prompt": prompt_text,
        "src": example['input'],
        "ref": example['output']
    }

dataset = dataset.map(Format_Bidirectional_Prompt)
print(f"✅ 데이터 준비 완료: {len(dataset)} samples")

In [ ]:
# 7. GRPO 학습 설정 및 실행
from trl import GRPOTrainer, GRPOConfig

# T4는 FP16 강제
if "T4" in gpu_name_short:
    print("⚠️ T4 Detected: Forcing FP16")

training_args = GRPOConfig(
    output_dir="./grpo-output",
    num_train_epochs=1,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    learning_rate=2e-6,
    
    # GRPO 설정
    num_generations=4,
    max_completion_length=200,
    beta=0.04,
    use_vllm=True,
    vllm_gpu_memory_utilization=VLLM_MEM,
    
    # 정밀도 (T4: FP16)
    fp16=True,
    bf16=False,
    
    # 로깅
    logging_steps=1,
    save_steps=100,
    save_total_limit=3,
    report_to="none",
    
    # Hub 업로드
    push_to_hub=True,
    hub_model_id=OUTPUT_MODEL_ID,
    hub_strategy="every_save",
)

trainer = GRPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
    reward_funcs=reward_function,
)

print("🚀 학습 시작...")
trainer.train()

In [ ]:
# 8. 모델 저장 및 업로드
trainer.save_model("./final_model")
trainer.push_to_hub()
print(f"✅ 업로드 완료: {OUTPUT_MODEL_ID}")